In [1]:
import os 
import sys

import pandas as pd

sys.path.append('../../')
import config

In [2]:
data = pd.read_csv(config.DATA_DIR + '/01-ais/01-data.csv', parse_dates=['time'])
test_data = pd.read_csv(config.DATA_DIR + '/00-raw/ais_test.csv', parse_dates=['time'])

In [3]:
data

,time,cog,sog,rot,heading,navstat,latitude,longitude,vesselId,portId,eta_hours
0,2024-01-01 00:00:25,284.0,0.7,0.0,88.0,Under way using engine,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f,214.993056
1,2024-01-01 00:00:36,109.6,0.0,-6.0,347.0,At anchor,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689,8731.990000
2,2024-01-01 00:01:45,111.0,11.0,0.0,112.0,Under way using engine,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19,32.970833
3,2024-01-01 00:03:11,96.4,0.0,0.0,142.0,At anchor,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126,8779.946944
4,2024-01-01 00:03:51,214.0,19.7,0.0,215.0,Under way using engine,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3,587.935833
...,...,...,...,...,...,...,...,...,...,...,...
1522059,2024-05-07 23:59:07,359.1,13.4,0.0,1.0,Under way using engine,52.19131,-5.82223,clh6aqawa0002gh0zypfa5dut,634c4de270937fc01c3a7417,5.014722
1522060,2024-05-07 23:59:08,12.3,17.1,0.0,13.0,Under way using engine,38.96142,-12.00502,61e9f3aeb937134a3c4bfe43,634c4de270937fc01c3a76a1,51.014444
1522061,2024-05-07 23:59:08,269.8,14.9,-1.0,270.0,Under way using engine,49.71372,-5.22042,61e9f43db937134a3c4c0169,634c4de270937fc01c3a787b,191.014444
1522062,2024-05-07 23:59:08,8.0,18.7,0.0,6.0,Under way using engine,38.27895,10.78280,61e9f469b937134a3c4c029b,61d3781293c6feb83e5eb73b,12.764444


In [4]:
data.vesselId.nunique()

688

In [5]:
# Check nunique last five days
unique_vessels = data[data.time>data.time.max() - pd.Timedelta(days = 5)].vesselId

In [6]:
unique_vessels.nunique()

487

In [7]:
test_data.vesselId.nunique()

215

In [8]:
test_data.vesselId.value_counts()

vesselId
61e9f466b937134a3c4c0273    360
61e9f403b937134a3c4c0011    360
61e9f3c8b937134a3c4bfee5    360
61e9f3bcb937134a3c4bfe91    360
61e9f399b937134a3c4bfdb7    360
                           ... 
61e9f432b937134a3c4c011b    128
61e9f39fb937134a3c4bfdcd    127
6326f5a6c46d6a20d22ca31e    125
61e9f45ab937134a3c4c021b    123
61e9f41cb937134a3c4c0091    121
Name: count, Length: 215, dtype: int64

### Train validation division
Lets use the last 5 days of the train data as validation data

In [9]:
data

,time,cog,sog,rot,heading,navstat,latitude,longitude,vesselId,portId,eta_hours
0,2024-01-01 00:00:25,284.0,0.7,0.0,88.0,Under way using engine,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f,214.993056
1,2024-01-01 00:00:36,109.6,0.0,-6.0,347.0,At anchor,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689,8731.990000
2,2024-01-01 00:01:45,111.0,11.0,0.0,112.0,Under way using engine,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19,32.970833
3,2024-01-01 00:03:11,96.4,0.0,0.0,142.0,At anchor,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126,8779.946944
4,2024-01-01 00:03:51,214.0,19.7,0.0,215.0,Under way using engine,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3,587.935833
...,...,...,...,...,...,...,...,...,...,...,...
1522059,2024-05-07 23:59:07,359.1,13.4,0.0,1.0,Under way using engine,52.19131,-5.82223,clh6aqawa0002gh0zypfa5dut,634c4de270937fc01c3a7417,5.014722
1522060,2024-05-07 23:59:08,12.3,17.1,0.0,13.0,Under way using engine,38.96142,-12.00502,61e9f3aeb937134a3c4bfe43,634c4de270937fc01c3a76a1,51.014444
1522061,2024-05-07 23:59:08,269.8,14.9,-1.0,270.0,Under way using engine,49.71372,-5.22042,61e9f43db937134a3c4c0169,634c4de270937fc01c3a787b,191.014444
1522062,2024-05-07 23:59:08,8.0,18.7,0.0,6.0,Under way using engine,38.27895,10.78280,61e9f469b937134a3c4c029b,61d3781293c6feb83e5eb73b,12.764444


In [10]:
train_val_data_divider = data.time.max().round(freq='D') - pd.Timedelta(days=5)

In [11]:
train_data = data[data.time<train_val_data_divider]
val_data = data[data.time>=train_val_data_divider]

In [12]:
scaling_factors = pd.Series([3 - i * 0.5 for i in range(len(val_data['time'].dt.normalize().unique()))], 
                            index=pd.date_range(val_data['time'].min().normalize(), val_data['time'].max().normalize(), freq='D'))
val_data['scaling_factor'] = val_data['time'].dt.normalize().map(scaling_factors)


/var/folders/sz/sv4w8zxs0l9g59yx7yjgl9580000gn/T/ipykernel_31115/409115132.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['scaling_factor'] = val_data['time'].dt.normalize().map(scaling_factors)


In [13]:
train_data.shape

(1461558, 11)

In [14]:
val_data.shape

(60506, 12)

In [15]:
val_data.head()

,time,cog,sog,rot,heading,navstat,latitude,longitude,vesselId,portId,eta_hours,scaling_factor
1461558,2024-05-03 00:02:06,195.1,18.4,1.0,195.0,Under way using engine,30.78370,-14.12051,631e3cc6192150e13fa41625,634c4de270937fc01c3a76dc,587.965000,3.0
1461559,2024-05-03 00:02:11,121.9,13.0,-1.0,125.0,Under way using engine,15.59584,-102.14670,61e9f39bb937134a3c4bfdbd,634c4de270937fc01c3a74ec,121.963611,3.0
1461560,2024-05-03 00:02:53,100.0,8.1,0.0,99.0,Under way using engine,32.93787,34.58350,61e9f441b937134a3c4c018b,634c4de270937fc01c3a76b1,-2121.048056,3.0
1461561,2024-05-03 00:04:23,15.0,0.0,0.0,115.0,Moored,35.38995,139.64783,6326ec84c46d6a20d22ca318,61d379f61366c3998241d8d2,66.926944,3.0
1461562,2024-05-03 00:05:56,140.1,0.0,0.0,306.0,Moored,40.71222,29.46758,61e9f470b937134a3c4c02cd,61d38259b7b7526e1adf3a41,-787.098889,3.0


In [16]:
train_data.to_csv(config.DATA_DIR+'/01-ais/02-train_data.csv', index = False)

In [17]:
val_data[['time', 'vesselId', 'latitude', 'longitude', 'scaling_factor']].to_csv(config.DATA_DIR+'/01-ais/02-val_data.csv', index = False)
val_data.to_csv(config.DATA_DIR+'/01-ais/02-val_data_with_features.csv', index = False)